## 📌 一、交叉熵的本质：衡量两个概率分布的“差距”

在多分类任务中，模型的输出通常是一个**概率分布**，即通过 softmax 得到的各类别概率。

而交叉熵就是衡量两个概率分布（真实标签和预测分布）之间“差异”的一种方式。

---

## 📘 二、公式详解

### ✅ 情况1：用 one-hot 标签时（常见）

设：
- 真实标签（one-hot 编码）为 $ \mathbf{y} = [y_1, y_2, ..., y_C] $，其中只有一个为 1，其他为 0；
- 模型输出（softmax 概率）为 $ \hat{\mathbf{y}} = [\hat{y}_1, \hat{y}_2, ..., \hat{y}_C] $；

则交叉熵损失定义为：

$$
\mathcal{L} = - \sum_{i=1}^{C} y_i \log(\hat{y}_i)
$$

因为只有 $ y_k = 1 $，其余为 0，所以上式简化为：

$$
\mathcal{L} = - \log(\hat{y}_k)
$$

👉 也就是说：只惩罚模型在**真实类别上的概率预测是否足够高**。

---

### ✅ 情况2：使用 soft label 或 label smoothing

如果真实标签不是 one-hot，而是一个概率分布，比如：

$$
y = [0.1, 0.8, 0.1]
$$

此时，交叉熵依然适用，表示两个概率分布之间的“差异”程度：

$$
\mathcal{L} = - \sum_{i=1}^{C} y_i \log(\hat{y}_i)
$$

---

## 🔢 三、与 softmax 联合使用

在多分类神经网络中，通常是：
1. 最后一层输出 logits（未归一化分数）
2. 对 logits 使用 softmax 转换成概率
3. 与真实标签计算交叉熵损失

例如，模型输出 logits 为：

$$
z = [1.2, 0.4, -0.5]
$$

softmax 得到：

$$
\hat{y}_i = \frac{e^{z_i}}{\sum_{j=1}^{C} e^{z_j}}
$$

再带入交叉熵公式。

💡 PyTorch 中的 `CrossEntropyLoss` 会**自动将 logits 做 softmax**，不用你自己手动加 softmax。

---

## 🎯 四、交叉熵的目标：最大化真实标签的预测概率

交叉熵损失越小，说明预测越“接近”真实标签的概率分布。

它本质上在做一件事：

$$
\text{最小化交叉熵} \quad = \quad \text{最大化预测中真实类别的概率}
$$

In [2]:
import torch
import torch.nn as nn

logits = torch.tensor([
    [2.0, 0.5, 0.3],   # 样本1
    [0.1, 2.1, 0.3],   # 样本2
    [0.2, 0.4, 1.9]    # 样本3
])

# 对应的真实标签（类别编号，不是 one-hot）
# 表示样本1是类别0，样本2是类别1，样本3是类别2
targets = torch.tensor([0, 1, 2])
loss_fn = nn.CrossEntropyLoss()
loss = loss_fn(logits, targets)

print("平均交叉熵损失：", loss.item())

平均交叉熵损失： 0.3146948516368866


In [3]:
torch.softmax(input=logits, dim=1)

tensor([[0.7113, 0.1587, 0.1299],
        [0.1041, 0.7689, 0.1271],
        [0.1299, 0.1587, 0.7113]])